In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import models

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
BATCH_SIZE=16
lr = 0.001
NUM_EPOCHS = 10

In [5]:
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
vgg16 = models.vgg16(pretrained=True)
vgg16.to(device)
print(vgg16)

/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
# change the number of classes 
vgg16.classifier[6].out_features = 10
# freeze convolution weights
for param in vgg16.features.parameters():
    param.requires_grad = False

In [8]:
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=lr, momentum=0.9)
# loss function
criterion = nn.CrossEntropyLoss()

In [10]:
def validate(model, test_loader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    for int, data in enumerate(test_loader):
        data, target = data[0].to(device), data[1].to(device)
        output = model(data)
        loss = criterion(output, target)
        
        val_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        val_running_correct += (preds == target).sum().item()
    
    val_loss = val_running_loss/len(test_loader.dataset)
    val_accuracy = 100. * val_running_correct/len(test_loader.dataset)
    
    return val_loss, val_accuracy

In [11]:
def fit(model, train_loader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for i, data in enumerate(train_loader):
        data, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_running_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()
    train_loss = train_running_loss/len(train_loader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_loader.dataset)
    
    return train_loss, train_accuracy

In [13]:
def load_checkpoint(checkpoint):
    vgg16.load_state_dict(checkpoint['vgg16_state_dict'])
    optim.load_state_dict(checkpoint['optim_state_dict'])
    return checkpoint['train_losses'], checkpoint['val_losses']

def save_checkpoint(checkpoint, model_path):
    torch.save(checkpoint, model_path)

In [14]:
load_model = False
file_path = f'/kaggle/working/vgg16.pth.tar'


train_losses , train_accuracy = [], []
val_losses , val_accuracy = [], []

if load_model:
    train_losses, val_losses = load_checkpoint(torch.load(file_path))

len(train_losses), len(val_losses)

(0, 0)

In [15]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    train_epoch_loss, train_epoch_accuracy = fit(vgg16, train_loader)
    val_epoch_loss, val_epoch_accuracy = validate(vgg16, test_loader)
    train_losses.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)
    end = time.time()
    
    checkpoint = {
        'vgg16_state_dict': vgg16.state_dict(), 'optimizer_state_dict': optimizer.state_dict(),
        'train_losses': train_losses, 'val_losses': val_losses
    }

    save_checkpoint(checkpoint, file_path)
    
    print(f'Epoch: [{epoch} | {NUM_EPOCHS}], Train Loss: {train_epoch_loss:.4f}, Train Acc: {train_epoch_accuracy:.2f} \
          Test Loss: {val_epoch_loss:.4f}, Test Accuracy: {val_epoch_accuracy:.2f} \
          Time Taken: {(time.time()-start_time)/60:.2f} mins')

Epoch: [0 | 10], Train Loss: 0.0451, Train Acc: 75.96           Test Loss: 0.0283, Test Accuracy: 84.47           Time Taken: 6.98 mins
Epoch: [1 | 10], Train Loss: 0.0257, Train Acc: 85.68           Test Loss: 0.0264, Test Accuracy: 85.70           Time Taken: 13.90 mins
Epoch: [2 | 10], Train Loss: 0.0185, Train Acc: 89.61           Test Loss: 0.0255, Test Accuracy: 86.68           Time Taken: 20.81 mins
Epoch: [3 | 10], Train Loss: 0.0131, Train Acc: 92.66           Test Loss: 0.0254, Test Accuracy: 87.10           Time Taken: 27.73 mins
Epoch: [4 | 10], Train Loss: 0.0086, Train Acc: 95.20           Test Loss: 0.0280, Test Accuracy: 86.95           Time Taken: 34.64 mins
Epoch: [5 | 10], Train Loss: 0.0060, Train Acc: 96.77           Test Loss: 0.0278, Test Accuracy: 87.34           Time Taken: 41.54 mins
Epoch: [6 | 10], Train Loss: 0.0039, Train Acc: 97.84           Test Loss: 0.0286, Test Accuracy: 87.66           Time Taken: 48.45 mins
Epoch: [7 | 10], Train Loss: 0.0027, Train